In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marathi-english-sum/translated_mar_eng 0.5.csv
/kaggle/input/marathi-english-sum/translated_mar_eng_sum.csv
/kaggle/input/marathi-english-sum/translated_mar_eng_text.csv
/kaggle/input/marathi-english-sum/translated_mar_eng 0.7.csv
/kaggle/input/marathi-english-sum/mar_eng_sum.csv
/kaggle/input/english-summarization-dataset/test_final_eng.csv
/kaggle/input/english-summarization-dataset/val_final_eng.csv
/kaggle/input/english-summarization-dataset/train_final_eng.csv
/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl


In [2]:
import pandas as pd
df_test_final_mar=pd.read_csv("/kaggle/input/english-summarization-dataset/test_final_eng.csv")
df_test_final_mar['text'][0]

'Hamburg was one of five cities left in the running, alongside Rome, Paris, Budapest and Los Angeles. But 51.6% of residents of the city voted no in a referendum on Sunday. The No camp argued that money earmarked for the Olympics could be better spent. German Olympics officials had picked Hamburg as their preferred candidate city ahead of Berlin. Germany has not hosted the Games since 1972 in Munich. Voters there turned down the chance to host the 2022 Winter Olympics two years ago. Almost two-thirds of voters in nearby Kiel, where sailing events would have been held, backed the Olympic proposal in Sunday\'s referendum. But turnout in the relatively small city of Kiel was under 32%, while 651,000 voters in Hamburg took part - half of the electorate. Hamburg\'s mayor Olaf Scholz was disappointed with the result: "That\'s a decision that we didn\'t want, but it\'s clear." The remaining cities vying for the chance to stage the 2024 Games include Budapest, Paris, Los Angeles and Rome. Crit

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# from transformers import AutoTokenizer

# model_checkpoint = "october-sd/pegasus-xsum-finetuned-en-sum"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

In [28]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/bart-large-xsum-finetuned-en-sum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [29]:
# from transformers import AutoTokenizer

# model_checkpoint = "siddharth57/t5-base-T-5-BASE-FINETUNED"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [30]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/304 [00:00<?, ?B/s]

In [31]:
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import torch
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score
import pandas as pd

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [36]:
def generate_summaries(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=512, max_target_length=128):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, num_beams=4, length_penalty=2.0, early_stopping=True)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [37]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")
def generate_summaries_indic(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=512, max_target_length=60):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, use_cache=True, num_beams=4, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [38]:
test_texts = df_test_final_mar['text'].tolist()  # Update column name if necessary
reference_summaries = df_test_final_mar['summary'].tolist()  # Update column name if necessary

In [39]:
generated_summaries = generate_summaries(test_texts, model, tokenizer, device)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [40]:
print(generated_summaries[0])
print(reference_summaries[0])

Voters in the German city of Hamburg have rejected a plan to host the 2024 Olympic Games, ending the chance for Germany to stage the Games for the first time in decades.
Residents of the German city of Hamburg have voted against hosting the 2024 Olympic and Paralympic Games.


In [41]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_summaries, generated_summaries)]

In [42]:
# Compute BLEU score
bleu_score = corpus_bleu(generated_summaries, [reference_summaries]).score

In [43]:
# Compute BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang="mr", device=device, batch_size=64)
bertscore_avg = F1.mean().item()

In [44]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 14.06
BERTScore: 0.7942


In [45]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 44.0666
Average rouge2 F1: 21.0138
Average rougeL F1: 35.5127


In [46]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 14.06
BERTScore: 0.7942


In [47]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 44.0666
Average rouge2 F1: 21.0138
Average rougeL F1: 35.5127


In [48]:
rouge1_avg = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
rougeL_avg = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 F1: {rouge1_avg:.2f}")
print(f"Average ROUGE-L F1: {rougeL_avg:.2f}")

Average ROUGE-1 F1: 0.44
Average ROUGE-L F1: 0.36


In [49]:
#mbart
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 14.06
BERTScore: 0.7942


In [50]:
#mbart
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 44.0666
Average rouge2 F1: 21.0138
Average rougeL F1: 35.5127


In [51]:
# #indicbart 
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [52]:
# #indicbart 
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [53]:
# #indicbart finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [54]:
# #indicbart finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [55]:
# #indicbart finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [56]:
# #indicbart finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [57]:
# #mt5 finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [58]:
# #mt5 finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")